In [7]:
import numpy as np
from qutip import Qobj, tensor, basis, ptrace
import pandas as pd

I = Qobj(np.eye(2))
X = Qobj(np.array([[0, 1], [1, 0]]))
Y = Qobj(np.array([[0, -1j], [1j, 0]]))
Z = Qobj(np.array([[1, 0], [0, -1]]))

def apply_pauli(pauli, n, i):
    result = [I] * n
    result[i] = pauli
    return tensor(result)

def frame(n, J, lamb_val):
    H = 0
    if n == 1:
        H = lamb_val[0][0] * X + lamb_val[1][0] * Y + lamb_val[2][0] * Z
        return H
    for i in range(n - 1):
        H += J * apply_pauli(X, n, i) * apply_pauli(X, n, i + 1)
        H += lamb_val[0][i] * apply_pauli(X, n, i) * apply_pauli(X, n, i + 1)
        H += lamb_val[1][i] * apply_pauli(Y, n, i) * apply_pauli(Y, n, i + 1)
        H += lamb_val[2][i] * apply_pauli(Z, n, i) * apply_pauli(Z, n, i + 1)
    for i in range(n):
        H += apply_pauli(Z, n, i)
        H += lamb_val[0][i] * apply_pauli(X, n, i)
        H += lamb_val[1][i] * apply_pauli(Y, n, i)
        H += lamb_val[2][i] * apply_pauli(Z, n, i)
    return H

def measureX(state, n):
    probx = []
    for i in range(n):
        rho_i = ptrace(state, i)
        plus = (basis(2, 0) + basis(2, 1)).unit()
        minus = (basis(2, 0) - basis(2, 1)).unit()
        prob_plus = (rho_i * plus * plus.dag()).tr()
        prob_minus = (rho_i * minus * minus.dag()).tr()
        probx.append((prob_plus, prob_minus))
    return probx

def measureY(state, n):
    proby = []
    for i in range(n):
        rho_i = ptrace(state, i)
        plus_y = (basis(2, 0) + 1j * basis(2, 1)).unit()
        minus_y = (basis(2, 0) - 1j * basis(2, 1)).unit()
        prob_plus_y = (rho_i * plus_y * plus_y.dag()).tr()
        prob_minus_y = (rho_i * minus_y * minus_y.dag()).tr()
        proby.append((prob_plus_y, prob_minus_y))
    return proby

def measureZ(state, n):
    probz = []
    for i in range(n):
        rho_i = ptrace(state, i)
        prob_0 = (rho_i * basis(2, 0) * basis(2, 0).dag()).tr()
        prob_1 = (rho_i * basis(2, 1) * basis(2, 1).dag()).tr()
        probz.append((prob_0, prob_1))
    return probz

def calFI(prob_j1, prob_j2, prob_j):
    fisher_info = 0
    delta_j = 0.02
    for (prob_plus_j1, prob_minus_j1), (prob_plus_j2, prob_minus_j2), (prob_plus_j, prob_minus_j) in zip(prob_j1, prob_j2, prob_j):
        if prob_plus_j != 0 and prob_minus_j != 0:
            f_plus = ((prob_plus_j1 - prob_plus_j2) / delta_j) ** 2 / prob_plus_j
            f_minus = ((prob_minus_j1 - prob_minus_j2) / delta_j) ** 2 / prob_minus_j
            fisher_info += f_plus + f_minus
        else:
            fisher_info += 0
    return fisher_info

def measureprob(state, n):
    prob_x = measureX(state, n)
    prob_y = measureY(state, n)
    prob_z = measureZ(state, n)
    return prob_x, prob_y, prob_z

def gendataset(numsamps):
    input_data = []
    output_data = []
    for _ in range(numsamps):
        n = 6
        J1 = 0.51
        J2 = 0.49
        J = 0.5
        lamb_c = [np.random.uniform(-1, 1, n-1) for _ in range(3)]
        lamb_l = [np.random.uniform(-1, 1, n) for _ in range(3)]
        lamb_val = [np.concatenate((lamb_c[i], lamb_l[i])) for i in range(3)]

        H1 = frame(n, J1, lamb_val)
        H2 = frame(n, J2, lamb_val)
        H = frame(n, J, lamb_val)

        eval1, evec1 = H1.eigenstates()
        eval2, evec2 = H2.eigenstates()
        eval, evec = H.eigenstates()

        gstate1 = evec1[0]
        gstate2 = evec2[0]
        gstate = evec[0]

        prob_x_j1, prob_y_j1, prob_z_j1 = measureprob(gstate1, n)
        prob_x_j2, prob_y_j2, prob_z_j2 = measureprob(gstate2, n)
        prob_x_j, prob_y_j, prob_z_j = measureprob(gstate, n)

        FI_x = calFI(prob_x_j1, prob_x_j2, prob_x_j)
        FI_y = calFI(prob_y_j1, prob_y_j2, prob_y_j)
        FI_z = calFI(prob_z_j1, prob_z_j2, prob_z_j)
        FI = FI_x + FI_y + FI_z

        input_data.append(np.concatenate(lamb_val).tolist())
        output_data.append(float(FI.real))
    
    return input_data, output_data

def save_datasets(input_data, output_data, input_file='input_data.csv', output_file='output_data.csv'):
    input_df = pd.DataFrame(input_data, columns=[f'lambda_{i}_{j}' for i in range(3) for j in range(len(input_data[0]) // 3)])
    output_df = pd.DataFrame(output_data, columns=['QFI'])
    input_df.to_csv(input_file, index=False)
    output_df.to_csv(output_file, index=False)
    print(f"Done! Datasets generated and saved to '{input_file}' and '{output_file}'.")

if __name__ == "__main__":
    num_samples = 3000  
    input_data, output_data = gendataset(num_samples)
    save_datasets(input_data, output_data)


Done! Datasets generated and saved to 'input_data.csv' and 'output_data.csv'.
